注：footnoteのリンクは壊れてます．対応する場所は手動で参照してください．


# 導入

この講座では，強化学習理論における2つのトピック (1) プランニングと(2) オンライン強化学習を取り扱う．これらのトピックは次のように定義される．

* プランニングとは，行動プラン，方策または行動そのものを，何らかの環境モデルを通して計算する問題を指す．
* オンライン強化学習とは，長期的な報酬の和を最大化する行動を，環境内で実際に行動しながら見つける問題を指す．

本講義では，これら全ての問題に対して，マルコフ決定過程 (Markov Decision Processes 略して MDP) を使って環境のシミュレーションモデルまたは環境自体を表現していく．そこでまず， MDP の定義から始めよう．


## マルコフ決定過程
マルコフ決定過程 (MDP) とは，確率的な状態遷移が生じる逐次意思決定問題を数学的に定義するモデルである． MDP は，状態集合，行動集合，状態間の確率的遷移のルール，報酬そして目的関数の５つから構築される．
本講義では，目的関数として基本的に割引報酬和の期待値 (リターン) を考える．

* 状態は抽象的な概念であり，陽に定義しない<a name="ref-1"></a>[<sup>[1]</sup>](#ref-1)．
状態集合は $\mathcal{S}$ と表記する．
簡単のため，有限集合とする<a name="ref-2"></a>[<sup>[2]</sup>](#ref-2)．
状態数は $\mathrm{S}$とする．
* 行動もまた抽象的な概念である．行動集合は $\mathcal{A}$ と表記する．
簡単のため，有限集合とする．
行動数は $\mathrm{A}$ と表記する．
* 状態 $s$ と $s'$ の間の (確率的) 状態遷移は，行動 $a$ を与えられた状態で行う事で生じる．
状態 $s$ と行動 $a$ に対し，あらたな状態 $s'$ に移動する確率をベクトルにまとめ， $P_a(s)$ と表記する<a name="ref-3"></a>[<sup>[3]</sup>](#ref-3)．
表記を簡単にするため，記法を乱用することになるが，このベクトルの $s'\in \mathcal{S}$ に該当する要素を $P_a(s,s')$ と表記する．これは，状態 $s$ で行動 $a$ を行ったとき，次状態 $s'$ に遷移する確率となる．
* 報酬は実数値で与えられ，状態 $s$ で行動 $a$ を行ったときの報酬を $r_a(s)$ と表記する．
状態と行動の数が有限であるため，報酬が閉区間 $[0,1]$ 内に属するとしても問題はなく，そのように仮定する<a name="ref-4"></a>[<sup>[4]</sup>](#ref-4)．

時刻 $t$ で行動 $A_t$ を行うとすると，無限に長い軌道 $S_0,A_0,S_1,A_1,...$ が得られる．ここで， $S_{t+1}$ は行動 $A_t$ を時刻 $t\ge 0$ で行った結果として生じる．また，行動 $A_t$ は過去の履歴 (後ほど定義) にのみ依存し， $S_{t+1}$ は $S_0,A_0,\dots,S_t,A_t$ が与えられたとき， $P_{A_t}(S_t)$ にのみ ($\mathbb{P}$-almost surelyで) 依存する．

$$
\begin{align}
\mathbb{P}(S_{t+1}=s|S_0,A_0,\dots,S_t,A_t) = P_{A_t}(S_t,s)\,.
\end{align}
$$

我々が考える問題のゴールは，軌道に沿ったリターンが最大化するような行動を選択するルールの発見である．
軌道に沿ったリターンは次のように定義される．

$$
R = r_{A_0}(S_0) + \gamma r_{A_1}(S_1) + \gamma^2 r_{A_2}(S_2) + \dots + \gamma^t r_{A_t}(S_t) + \dots
$$

ここで $\gamma \in [0,1)$ は割引率と呼ばれる．
正式には， (割引) MDP は $5$つの組 $M = (\mathcal{S},\mathcal{A},P,r,\gamma)$ で定義される．ここで $P=(P_a(s))_{s,a}$ と $r=(r_a(s))_{s,a}$ は確率遷移ベクトルと報酬の組である.



## 割引について

割引率 $\gamma$ は，現在の報酬に対して，将来の報酬の重要度を低く割り当てる．
これによって，無限遠の報酬を考える必要がなくなる．
試しに上記のリターンの和を $H\ge 0$ までの項で打ち切る場合を考えてみよう．
報酬の定義により，通常のリターンと打ち切られたリターンの差は $0$ 以上かつ

$$
\gamma^H \Big[r_{A_H}(S_H)+\gamma r_{A_{H+1}}(S_{H+1})+\dots \Big]\le
\gamma^H \sum_{s\ge 0} \gamma^s =
\frac{\gamma^H}{1-\gamma}
$$

となる．ここでは[等比数列の和の公式](https://ja.wikipedia.org/wiki/%E7%AD%89%E6%AF%94%E6%95%B0%E5%88%97)を用いた．
これは $H$ で報酬を打ち切った場合の誤差であり，
$H$が次を満たす値の場合，この打ち切り誤差の上限が $\varepsilon$ 以下になる．

$$
H \geq \underbrace{\frac{\ln \left(\frac{1}{\varepsilon(1-\gamma)}\right)}{\ln (1 / \gamma)}}_{H_{\gamma, \varepsilon}^*} .
$$

これが意味するのは， $\varepsilon$ の誤差を無視したとき，最初の $H$ 時刻分の報酬を考えるだけで，すでにリターンは最大化されているということだ．<a name="ref-5"></a>[<sup>[5]</sup>](#ref-5)
この上限を満たす $H$ は $\varepsilon$ だけでなく $\gamma$ にも依存することに注意してほしい．ある固定された $\varepsilon$ に対し，このような $H$ をエフェクティブホライゾンと呼ぶ．ただし，実際には次の緩いバウンドのことをエフェクティブホライゾンと呼ぶことも多い．

$$
H_{\gamma, \varepsilon}:=\frac{\ln \left(\frac{1}{\varepsilon(1-\gamma)}\right)}{1-\gamma}
$$

これは$H_{\gamma, \varepsilon}^*$の上界であり，2つの量の差は$\gamma\to 1$のときに$o(1-\gamma)$のオーダーであることに注意してほしい．
つまり，$H_{\gamma, \varepsilon}^*$は$\gamma \to 1$のときに$H_{\gamma, \varepsilon}$とfirst-orderで同じである．
この分野では多くの場合で$\gamma\to 1$の状況に興味があるので，以降では$H_{\gamma, \varepsilon}$を使うことにする．

将来の報酬を割り引くというのはしっくりとこないかもしれない．割引率とは一体なんなのだろうか？
割引率はしばしば，「どれくらい先の時刻まで考慮したリターンの最大化をするべきか，そしてどの程度の精度の最適行動を望むか」から逆算して決められる．
しかしながら，割引されたリターンは意思決定問題の本質を正確には捉えていないことがある．
実際，他にも目的関数 (または異なる問題設定) を考えることが出来る．例えば，有限時間非割引 (または割引) 問題，無限時間非割引 ("総報酬") 問題，無限時間平均報酬問題などがある．それぞれに長所と短所があり，これらの関係については後の講義で触れる．
当面は，無限時間割引問題を考えよう．数学的に簡単で美しいからだ．また，多くの結果は(いくつかの追加条件や小さな変更で)他の問題設定で成り立つ ．



## 方策

方策は，与えられた過去の履歴をもとに行動を選択するルールである．
時刻 $t\geq 0$ のおける過去の履歴とは

$$
H_t = (S_0,A_0,\dots,S_{t-1},A_{t-1},S_t)
$$

で定義される．つまり，時刻 $t$ までのすべての状態と行動のペアだ．
方策は確率的なものも許す．
よって方策は，無限長の系列 $\pi = (\pi_t)_{t\ge 0}$ で，各 $\pi_t$ は過去の履歴から行動空間上の確率分布への写像となる．有限集合 $X$ に対し， $\mathcal{M}_1(X)$ を $X$ 上の確率分布の集合とする．
各確率分布は，$X$ の要素に対する確率の割り当て方で一意に定まる．
よって， $|X|$ 次元のベクトルとして表すことも出る(つまり，$x\in X$ 要素が $x$ の確率を表す)．
もし $p\in \mathcal{M}_1(X)$ ならば，要素 $x$ の確率を $p_x$ と $p(x)$ の両方で表す (状況によって適宜使い分ける)．

この記法を用いて， $\pi$ の $t$ 番目の"ルール" (このルールを用いて $t$ 時刻での行動を決める) を

$$
\pi_t: \mathcal{H}_t \to \mathcal{M}_1(\mathcal{A})
$$

と書くことが出来る．ただし，

$$
\mathcal{H}_t = (\mathcal{S} \times \mathcal{A})^{t-1} \times \mathcal{S}
$$

である．ここで $\mathcal{H}_0 = \mathcal{S}$ に注意しておこう．

直感的には，「方策 $\pi$ に従う」というのは，時刻 $t\ge 0$ において行動 $A_t$ が $\pi_t(H_t)$ からサンプルされるということである．言い換えると， $A_t=a$ の確率は $\pi_t(H_t)(a)$ と言える．
 $\pi_t(H_t)(a)$ と書くのは面倒なので， $\pi_t(a|H_t)$ と書くことにする．
つまり，方策 $\pi$ に従っているとき，時刻 $t\ge 0$ で ($\mathbb{P}$-almost surely)


$$
\begin{align}
\mathbb{P}(A_t=a|H_t) = \pi_t(a|H_t)
\end{align}
$$

である．

## 初期状態分布と軌道の分布について

MDPで方策に従って行動を選択する状況を考えよう．このとき，初期状態分布 $\mu\in \mathcal{M}_1(\mathcal{S})$ と方策に依存して，無限長の状態行動ペアからなる軌道の集合

$$
T = (\mathcal{S}\times \mathcal{A})^{\mathbb{N}}
$$

の上の分布で，式(1) と (2) そして

$$
\begin{align}
\mathbb{P}(S_0=s) = \mu(s)\,, \qquad s\in \mathcal{S}
\end{align}
$$

が各時刻 $t\ge 0$ において成り立つようなものが一意に定まる<a name="ref-6"></a>[<sup>[6]</sup>](#ref-6)

実際には，この分布はより大きな確率空間上の分布と考えることもできるが，その場合は一意に定まらない．
無限長の状態行動ペアからなる軌道の集合 $T$ 上の分布について話すときには，「この分布は，方策とMDPの対から誘導される正則確率空間上の分布である」という．<a name="ref-7"></a>[<sup>[7]</sup>](#ref-7)


そのような確率分布 $\mathbb{P}$ が初期状態分布 $\mu$ と方策 $\pi$ に依存することを強調するため，しばしば $\mathbb{P}_\mu^\pi$ と書く．しかし，文脈から方策と初期分布が明らかなら， $\mu$ と $\pi$ を省略する．
必要ならば， $\mathbb{P}$ と $\mathbb{P}_\mu^\pi$ に対応する期待値を，それぞれ $\mathbb{E}$ と $\mathbb{E}_\mu^\pi$ で表記する．

軌道 $\tau = (s_0,a_0,s_1,a_1,\dots)\in T$ が出る確率はいくらだろうか?
$h_t$ を $(s_0,a_0,\dots,s_{t-1},a_{t-1},s_t)$ としよう． $H_t = (S_0,A_0,\dots,S_{t-1},A_{t-1},S_{t})$ であったことを思い出してほしい．
[確率のチェインルール](https://ja.wikipedia.org/wiki/%E6%9D%A1%E4%BB%B6%E4%BB%98%E3%81%8D%E7%A2%BA%E7%8E%87)を繰り返し適用し，

$$
\begin{align*}
\mathbb{P}(&H_t=h_t)\\
&= \mathbb{P}(S_0=s_0,A_0=a_0,S_1=s_1,\dots,S_t=s_t)\\
&= \mathbb{P}(S_t=s_t|H_{t-1}=h_{t-1},A_{t-1}=a_{t-1}) \mathbb{P}(H_{t-1}=h_{t-1},A_{t-1}=a_{t-1}) \\
&= P_{a_{t-1}}(s_{t-1},s_t) \mathbb{P}(H_{t-1}=h_{t-1},A_{t-1}=a_{t-1})  \tag{by 式 (1)}\\
&= P_{a_{t-1}}(s_{t-1},s_t) \mathbb{P}(A_{t-1}=a_{t-1}|H_{t-1}=h_{t-1}) \mathbb{P}(H_{t-1}=h_{t-1})\\
&= P_{a_{t-1}}(s_{t-1},s_t) \pi_{t-1}(a_{t-1}|h_{t-1}) \mathbb{P}(H_{t-1}=h_{t-1}) \tag{by 式 (2)}\\
& \;\; \vdots \\
&= P_{a_{t-1}}(s_{t-1},s_t) \pi_{t-1}(a_{t-1}|h_{t-1})
\times \dots\times P_{a_0}(s_0,s_1) \pi_{0}(a_0|s_0) \mathbb{P}(S_0=s_0) \\
&= P_{a_{t-1}}(s_{t-1},s_t) \pi_{t-1}(a_{t-1}|h_{t-1})
\times \dots\times P_{a_0}(s_0,s_1) \pi_{0}(a_0|s_0) \mu(s_0) \tag{by 式 (3)}
\end{align*}
$$

を得る．項を整理し，

$$
\mathbb{P}(H_t=h_t)
= \mu_0(s_0) \left\{ \Pi_{i=0}^{t-1} \pi_i(a_i|h_i)\right\} \, \left\{ \Pi_{i=0}^{t-1} P_{a_i}(s_i,s_{i+1})\right\}
$$

が得られる．同様に，

$$
\mathbb{P}(H_t=h_t,A_t=a_t)
= \mu_0(s_0) \left\{ \Pi_{i=0}^{t} \pi_i(a_i|h_i)\right\} \, \left\{ \Pi_{i=0}^{t-1} P_{a_i}(s_i,s_{i+1})\right\}
$$

が成り立つ．

## 価値関数，最適価値関数，そして目的関数

MDP $M$ において，方策 $\pi$ で初期状態分布 $\mu$ から初期状態をサンプルしたときの割引期待報酬和の期待値，つまり期待リターンは

$$
v^\pi(\mu) = \mathbb{E}_\mu^\pi \left[ R \right]
$$

と書ける．
$\mu=\delta_s$ (ここで$\delta_s$ はディラックのδ関数で， $s$ でのみ $1$ となる確率分布) のとき，$v^\pi(s)$ と代わりに書くことにする．
この関数は各状態に対して定義できるので， $v^\pi$ は $\mathcal{S}$ 内の各状態に対し価値を割り当てる関数と考えることが出来る．この関数は方策 $\pi$ の *価値関数* と呼ばれる．
もしも MDP について言及する必要があるなら，"MDP $M$ での"と付け加え， そのMDP への依存性を $v^\pi_M$ として表す．

状態 $s\in \mathcal{S}$ において達成可能な最も高い価値は，すべての方策に関する最適化を行うことで計算ができて，

$$
v^*(s) = \sup_{\pi} v^\pi(s)
$$

で与えられる．ここで， $v^*: \mathcal{S}\to \mathbb{R}$ を状態空間上の関数として見たとき，これを *最適価値関数* と呼ぶ．
方策は， $v^\pi(s)=v^*(s)$ を満たすとき，状態 $s$ において *最適* であるという．もしもすべての状態で最適なら，方策は *一様最適* であるという．通常，一様最適な方策を探す方法について興味があるため，今後は一様最適と言わず，最適ということにする．

以降では次の疑問について考えていこう．

<center>
MDPが与えられたとき，最適方策を <em>効率よく計算する</em> にはどうすればよいか？
</center>


## プランニング = 計算

最適方策の計算は，「期待リターンを最大化するルール」を計算するプランニングの問題と見ることが出来る．これはアルゴリズム問題である．
ここでの*入力*は，最も簡単な場合，状態遷移確率と報酬を記述する巨大な表 (または，多数の表) である．ここで私たちが興味があるのは，この表をもとになんらかの最適方策 (の記述) を出力するアルゴリズムである．これまで述べてきたような，特に制限を課さない方策を表現するには，無限長の記述が必要そうであり，最適方策を計算する効率的なアルゴリズムは現実的でないように聞こえるかもしれない．しかし，すぐに説明するように，私たちが考えている有限の MDP の場合は，方策を簡潔に表すことが出来る<a name="ref-8"></a>[<sup>[8]</sup>](#ref-8)．
つまり，最適方策を探すアルゴリズムは意味があるのだ．

他のアルゴリズムの問題と同様に，我々が扱う主な疑問は 「 MDP を解く (=最適方策を見つける) のに必要な計算量はいくらか？」である．必要な計算量は状態と行動の数に従って増加することが予想できるだろう．実際，入力のサイズすらもこれらに依存する．
多くの場合，現実的な問題で見つかる MDP が，無限とは言わずとも，莫大な大きさの状態と行動の数を持つ．
もしも実際に状態と行動の数に従って計算量が増えるなら，計算量自体を考える必要はあるのだろうか？
答えはイエスだ．

* すべての MDP が常に巨大なわけではない．そのため，小さい MDP を解くのにどの程度の計算量が必要か理解することが役立つかもしれない．
小さな MDP を解くのに効率的な解法は，大きな MDP を解くための解法のベンチマークとなる．
* また，もしも MDP が大きいとしても，それを小さな MDP で近似できるかもしれない．そのような場合，小さな MDP のための解法も役立つ．
* 小さな MDP を効率的に解く方法は，いくつかの変更を加えることで，大きな MDP を解くための解法に一般化できるかもしれない．

ここまでの説明では，「小さい」・「大きい」という意味についてよく分からないと思われる．簡単に言うと，「小さい」というのは， MDP (および 方策) を定義する (状態遷移確率と報酬の) 表がコンピュータのメモリに充分収まるような問題で，「大きい」とはその他すべての問題を指す．

## その他雑多な注釈

### 無限長の軌道の確率?

上記の計算によって，無限長の軌道 $\tau = (s_0,a_0,s_1,a_1,\dots)$ の確率は

$$
\begin{align*}
\mathbb{P}(S_0=s_0,A_0=a_0,S_1=s_1,A_1=a_1,\dots)
&= \mu(s_0) \times \pi_0(a_0|h_0) \times P_{a_0}(s_0,s_1) \\
& \qquad\quad\;\; \times \pi_1(a_1|h_1) \times P_{a_1}(s_1,s_2) \\
& \qquad\quad\;\; \times \cdots \\
& \qquad\quad\;\; \times \pi_{t}(a_t|h_t) \times P_{a_t}(s_t,s_{t+1}) \\
& \qquad\quad\;\; \times \cdots
\end{align*}
$$

となると思うかもしれない．ただし，以前と同様に $h_t = (s_0,a_0,\dots,s_{t-1},a_{t-1},s_t)$ である．
しかしながら，これはうまくいかない．この軌道において，もし各行動の確率が $1/2$ だったとすると，右辺の無限個の積はゼロになってしまう!
であれば，$\mathbb{P}$ は一体全体，どうやって *定義* されるのか? そもそもそのような分布は存在するのか? もし存在するとし，上のような軌道にゼロの確率を割り当てるなら，すべての無限長の軌道にゼロの確率を割り当ててしまうのではないか?
このような無限の世界を歩くときには，注意深く進まなければならない!
この謎を解決するには，1. 測度論に基づいた確率を考える 2. そもそも無限長の軌道に基づいて定義されるものをすべて諦める（例えばリターン $R= \sum_{t\ge 0}\gamma^t r_{A_t}(S_t)$である）のどちらかを選択する必要がある． 測度論に基づいた確率の代わりとしては，すべてを極限で定義する (そして，常に無限長の軌道の有限部分のみの期待値をとる) 方法がある．しかし，これは面倒なので，測度論に基づいた方法を選ぶことにする．それについては，次の講義で説明しよう．

### Markovとは?
式 $(1)$ は，次状態を考えるだけなら，履歴の中の最後の状態と行動のみを考えるだけでよいということを意味する．これは，マルコフ (Markov) 性と呼ばれる．より一般的に，確率過程の一つであるマルコフ連鎖も似たような性質を持っている．

### Bellman's curse of dimensionality
初期の理論において重要な貢献をしたリチャード・ベルマン (Richard Bellman) は「次元の呪い」という語を作った．これでベルマンが意味したのは次のようなことである． MDP を使って現実の意思決定問題をモデル化すると，状態空間がいくつか ($d>0$ としよう) の集合の積 $\mathcal{S} = \mathcal{S}_1 \times \dots \times \mathcal{S}_d$ で表せる場面がよくある．もし各集合 $\mathcal{S}_i$ がたった2つ (!) の要素から成っていたとしても，状態空間は少なくとも $2^d$ 個の要素からなる．これは $d$ の関数として指数的に増大する．
よって，どのようなアルゴリズムも，すべての状態を *列挙* するという，たったそれだけのことが計算に含まれる場合，非常に長い計算が必要になる「呪い」にかかっている．
すでに説明したように，状態空間・行動空間ともに小さい場合を私たちは考えるが，主な主眼は，そうでない場合にどうやって *呪いを解消するか*だ．
しかしながら，後々説明するように，最悪の場合にはそれは不可能であることを明記しておく．どんな解決方法も，問題を修正する (目的関数を変更する，利用可能な入力を変更する，またはその両者) 必要がある．

### すべての状態で同じ行動空間?

本章では MDP がすべての状態において同じ行動空間を仮定した．これは，状態 $s_1$ における行動 $a_1$ が 状態 $s_2$ での行動 $a_1$ と何か関係があるという間違った印象を与えたかもしれない． (例えば， 報酬や次の状態の確率分布が同じ，または似ているといったような) しかし， MDP の定義を見ると分かるように，そういった仮定は全くしていない．

MDP の記述としてよりよいものは，集合 $Z$ とその集合 $Z$ 上の同値関係，または同じ意味だが，同値類で $Z$ を分割する方法だ．
ここで， $Z$ は可能な状態と行動のペアの集合だと考える．同値関係はどのペア達が同じ状態を持つか定める．つまり，もし $z_1$ と $z_2$ が同じ同値類に属するなら，それらは同じ状態を持つ．
そして，各 $z\in Z$ について， MDP は次の同値類 (状態) に遷移する確率と報酬を定める．
この方法は数学的な面からは魅力的だが，一般的ではないうえ，他の参考文献との対応を付けるのに苦労するだろう．
本章で扱った記法は，状態間で行動が必ずしも同じではないと頭に入れておく不便さを除けば，かなり分かりやすいし簡潔だ．

この「（行動の）共有問題」を避けるための，論文などでよく使われる他のバリエーションだと，各状態 $s$ に対して可能な行動の集合 $\mathcal{A}(s)$ があり，それらが集合間で排他的であるとする方法だ．
これは，行動数が各状態で変わってもいいという利点がある．しかしこの小さな利点を除き，私たちが使った記法は簡単で，他の洗練された記法を使ってもあまりいいことはない．

### 状態が観測可能?

多くの現実的な問題において，その問題が MDP でうまく近似可能であるかを事前に知っていることはない．
MDP に欠けている最も重大な点は，状態が計測不可能で，行動を決定するのに状態を使えないかもしれないという点だ．
当面はこの問題を脇に置いておくが，折に触れてこの問題に立ち返る.
その理由は，簡単な問題からスタートして，少なくとも感覚的に，状態情報を利用できる場合に最適方策を見つける方が，そうでない場合に最適方策を見つけるより簡単だと感じてもらうためだ．状態情報を利用できる場合にどうやって (そしてどの程度効率的に) 最適方策を見つけられるか学び，より複雑な設定の問題がこの場合に帰着できないか，または共通のパターンが存在しないかといったことを考える．

### 記法について

なぜ $r(s,a)$ ではなく $r_a(s)$ といった記法を使ったのか? また，なぜ $P(s'|s,a)$ という記法ではなく $P_a(s)$ や $P_a(s,s')$ を使ったのか？
これらの記法にはそれぞれ長短あるが，すべての状況に最も適合することはない．
私たちが用いた記法の利点だが，後々，私たちは $a$ を $\pi$ に置き換える．ここで， $\pi$ は特別な方策 (無記憶定常マルコフ方策)である．そうしたとき， $r_\pi$ ($s$ を除いてある) と $P_\pi$ (確率行列!) が非常に便利になる．

より大きな問題 (論争) は，なぜ $s$ を状態に， $a$ を行動に用いるかだ．でも，それは元の単語から来てるんじゃないか? うーん，制御理論を専門とする人たちは同意しないだろう．彼/女らは $x$ を状態に $u$ を行動に用いる．Maxim Raginskyから聞いたところによると，それらはロシア語の略語から来ているらしい．なので，少なくともここで使っている記法と同じくらいには正当性があるらしい． (もしあなたがロシア語を話すなら…．そうじゃないなら，ロシア語を勉強したらどう？) Dimitri Bertsekasは $i,j$ 等を状態に用いる．有限の行動空間を取り扱う場合には，それでも問題はない．

### ランダム報酬

著者によっては (例えば，この講義の著者の論文や本) ，報酬を確率的 (確率分布から毎回サンプルされる) な報酬とする．
これは，確率的なシミュレータを用いて，シミュレーションをしながらよい方策やプランを探すときには筋が通っている．
しかしながら，計算の目的を説明するにあたっては， (良設定な) 確率的な報酬は無視できる．
ここで，良設定な確率的報酬とは，現在の状態 $s$ と行動 $a$ までの任意の履歴で条件付けられたとき，条件付き期待値が $(s,a)$ のみに依存するものだ．

## 参考文献

MDP に関する本は次のものが定番だ．

Puterman, Martin L. 2005. Markov Decision Processes (Discrete Stochastic Dynamic Programming). Wiley-Interscience



<a name="ref-1"></a>1. [^](#ref-1) 状態集合が有限集合の場合，各要素に1から順に整数を割り当て，整数の集合とみなしてもまったく差し支えない．そのため，状態を陽に定義する必要はない．

<a name="ref-2"></a>2. [^](#ref-2) 状態集合を有限次元ユークリッド空間のコンパクト部分集合に拡張するのは特に問題は生じない．一方，行動空間を有限次元ユークリッド空間のコンパクト部分集合に拡張する場合， (後ほど出てくる) 最適方策の定義がうまくいかない． MDP 自体に仮定を置くか，可測集合族を拡張する必要がある．前者の詳細は末尾で引用されているPutermanの本に書いてあるため，そちらを参照してほしい．後者については， Bertsekas & Shreve の [Stochastic Optimal Control](http://web.mit.edu/dimitrib/www/soc.html) に書いてあるため，そちらを参照してほしい．

<a name="ref-3"></a>3. [^](#ref-3) これは次のように考えてほしい．まず，状態 $s$ で行動 $a$ を行ったとき，次状態 $s'$ に遷移する確率を $P_{a} (s, s')$ としよう．これを並べて作ったベクトルが $P_a (s)$ となる．ただし，次状態の並びに関しては，事前に順番を決めておき，その順番に従うものとする．

<a name="ref-4"></a>4. [^](#ref-4) 有限でない場合は報酬関数に適切な仮定を導入する必要がある．詳しくはPutermanの本を参照してほしい．

<a name="ref-5"></a>5. [^](#ref-5) 実際，右辺を $\gamma^H / (1-\gamma)$ に代入し， $\gamma$ を横軸にプロットしてみるとよい．かならず $\varepsilon$ 以下になっていることが確認できる．

<a name="ref-6"></a>6. [^](#ref-6) 状態空間と行動空間を離散と考えているので一意になる．一般の場合には一意にならないかもしれないが，普通に出くわすような空間では一意になることが多い．詳細については，確率過程の教科書にあたって欲しい．

<a name="ref-7"></a>7. [^](#ref-7)
この話は[Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)の4.6章を参考にするといい気がする．

<a name="ref-8"></a>8. [^](#ref-8) 事前知識が無い場合，ここはいまいち何を言っているか分からないと思われるので，少し注釈を加える．方策は，状態から行動空間上の確率分布への写像からなる無限長の系列 $\pi = (\pi_t)_{t \geq 0}$ であった．そのため，無限個の写像 (つまり，無限長の記述) が必要に思える．しかし実は，ある写像 $\pi^*$ が存在し， $\pi_1 = \pi_2 = \cdots = \pi^*$ となるような $\pi$ が最適であることが証明できる．
